In [5]:
# %% [markdown]
# # CAPTCHA Solver (OpenCV + NumPy + PIL)
# This notebook solves the "You Can Hire Some Indians Here" task without PyTorch or Tesseract.

# %% 
import cv2
import numpy as np
from PIL import Image
import os
import csv

# === Paths ===
SAMPLE_DIR = "sample_images"      # sample labeled images for template creation
TEMPLATE_DIR = "templates"        # folder to save character templates
TEST_DIR = "test_images"          # test images folder
OUTPUT_CSV = "submission.csv"     # final CSV output

# Create template folder if it doesn't exist
os.makedirs(TEMPLATE_DIR, exist_ok=True)

# %% [markdown]
# ## 1. Generate character templates from sample images
# For each labeled sample image, we segment letters and save each as a template.
# Filenames must include the label (e.g., `82tcrkeu.png`).

# %%
def preprocess(img):
    """Preprocess image: grayscale, binarize, invert"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, th = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
    return cleaned

def segment_letters(binary_img):
    """Return bounding boxes of letters sorted left-to-right"""
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(cnt) for cnt in contours]
    return sorted(boxes, key=lambda b: b[0])

# Example: generate templates from labeled samples
for fname in os.listdir(TEST_DIR):
    if fname.endswith(".png"):
        label = os.path.splitext(fname)[0]  # e.g., '82tcrkeu'
        img = cv2.imread(os.path.join(TEST_DIR, fname))
        bin_img = preprocess(img)
        boxes = segment_letters(bin_img)
        if len(boxes) != len(label):
            print(f"Warning: {fname} box count {len(boxes)} != label length {len(label)}")
            continue
        for i, (x, y, w, h) in enumerate(boxes):
            crop = bin_img[y:y+h, x:x+w]
            char_label = label[i]
            save_path = os.path.join(TEMPLATE_DIR, f"{char_label}.png")
            cv2.imwrite(save_path, crop)

print("✅ Templates generated in:", TEMPLATE_DIR)

# %% [markdown]
# ## 2. Load templates
# Load saved character templates for matching

# %%
templates = {}
for f in os.listdir(TEMPLATE_DIR):
    if f.endswith(".png"):
        char = f.split(".")[0]
        img = cv2.imread(os.path.join(TEMPLATE_DIR, f), 0)
        templates[char] = img
print(f"Loaded {len(templates)} character templates.")

# %% [markdown]
# ## 3. CAPTCHA Solver Functions

# %%
def match_char(img_crop, templates):
    """Match a single character crop to the best template using normalized correlation"""
    best_char = None
    best_score = -1
    for char, tmpl in templates.items():
        tmpl_resized = cv2.resize(tmpl, (img_crop.shape[1], img_crop.shape[0]))
        res = cv2.matchTemplate(img_crop, tmpl_resized, cv2.TM_CCOEFF_NORMED)
        _, score, _, _ = cv2.minMaxLoc(res)
        if score > best_score:
            best_score = score
            best_char = char
    return best_char

def solve_captcha(img_path, templates):
    """Return the predicted CAPTCHA string"""
    img = cv2.imread(img_path)
    bin_img = preprocess(img)
    boxes = segment_letters(bin_img)
    answer = ""
    for x, y, w, h in boxes:
        crop = bin_img[y:y+h, x:x+w]
        char = match_char(crop, templates)
        answer += char if char else "?"
    return answer

# %% [markdown]
# ## 4. Solve all test images and generate `submission.csv`

# %%
with open(OUTPUT_CSV, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename","answer"])
    
    for fname in sorted(os.listdir(TEST_DIR)):
        if fname.endswith(".png"):
            answer = solve_captcha(os.path.join(TEST_DIR, fname), templates)
            writer.writerow([fname, answer])
            print(fname, "->", answer)

print("✅ CSV saved:", OUTPUT_CSV)

✅ Templates generated in: templates
Loaded 0 character templates.
test_000.png -> ??????????????????????????????????
test_001.png -> ???????????????????????????????????????????
test_002.png -> ?????????????????????????????
test_003.png -> ??????????????????????????????????????????
test_004.png -> ??????????????????????
test_005.png -> ??????????????????
test_006.png -> ?????????????????????????????????
test_007.png -> ??????????????????????????????????????????????????????????????????????????????
test_008.png -> ????????????????????????????
test_009.png -> ???????????????????
test_010.png -> ?????????????
test_011.png -> ???????????????????????????????????????????????????????????????
test_012.png -> ????????????????????????????????????????????????????????????????????????????????????
test_013.png -> ????????????????????????????
test_014.png -> ????????????????????????????????????
test_015.png -> ????????????????????????????
test_016.png -> ?????????????????????????????????????
test_017.p

In [6]:
# Set your test images folder here
TEST_DIR = "test_images"  # <-- make sure this folder exists and contains .png files

# Check if folder exists
if not os.path.exists(TEST_DIR):
    print(f"⚠️ Warning: Test folder '{TEST_DIR}' not found. Creating it now.")
    os.makedirs(TEST_DIR)
    print(f"📂 Created empty folder '{TEST_DIR}'. Place your CAPTCHA test images here.")

# Generate submission CSV
with open(OUTPUT_CSV, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename","answer"])
    
    for fname in sorted(os.listdir(TEST_DIR)):
        if fname.endswith(".png"):
            answer = solve_captcha(os.path.join(TEST_DIR, fname), templates)
            writer.writerow([fname, answer])
            print(fname, "->", answer)

print("✅ CSV saved:", OUTPUT_CSV)


test_000.png -> ??????????????????????????????????
test_001.png -> ???????????????????????????????????????????
test_002.png -> ?????????????????????????????
test_003.png -> ??????????????????????????????????????????
test_004.png -> ??????????????????????
test_005.png -> ??????????????????
test_006.png -> ?????????????????????????????????
test_007.png -> ??????????????????????????????????????????????????????????????????????????????
test_008.png -> ????????????????????????????
test_009.png -> ???????????????????
test_010.png -> ?????????????
test_011.png -> ???????????????????????????????????????????????????????????????
test_012.png -> ????????????????????????????????????????????????????????????????????????????????????
test_013.png -> ????????????????????????????
test_014.png -> ????????????????????????????????????
test_015.png -> ????????????????????????????
test_016.png -> ?????????????????????????????????????
test_017.png -> ????????????????
test_018.png -> ???????????????????????????